<a href="https://colab.research.google.com/github/juliuserbach/Semantic-Features/blob/master/Mask_R_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ftaubner/semantic_features_detection.git

Cloning into 'semantic_features_detection'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1022 (delta 0), reused 0 (delta 0), pack-reused 1017
Receiving objects: 100% (1022/1022), 113.85 MiB | 49.61 MiB/s, done.
Resolving deltas: 100% (606/606), done.


In [3]:
!wget --no-check-certificate "https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199865&authkey=ALxjJOloi9Wjk80" -O mapillary_vistas_sample.zip

--2020-03-28 18:26:56--  https://onedrive.live.com/download?cid=EA356294C6263A37&resid=EA356294C6263A37%2199865&authkey=ALxjJOloi9Wjk80
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yijzyw.am.files.1drv.com/y4mSx0080F1_c9GEYoDRAv2tmAH_RCA7xTVASpvRv00t2z8dcVj6ACxaYoNTfttxUD90yqGZLTRg0FUv4Ag9Qt6qW40rXlhzS8y9iNkDn79LXAnaoFBNwAeXlVWjU6tT4zCR5tHHVwbZJcs9W7yKNAeqLcTvLCSbGZnEeMIUMjgaAtBb6G14qIFobbC7eU_3zUYkE2cs2sgGp1Mut9LAkT8SQ/mapillary_vistas_sample.zip?download&psid=1 [following]
--2020-03-28 18:26:57--  https://yijzyw.am.files.1drv.com/y4mSx0080F1_c9GEYoDRAv2tmAH_RCA7xTVASpvRv00t2z8dcVj6ACxaYoNTfttxUD90yqGZLTRg0FUv4Ag9Qt6qW40rXlhzS8y9iNkDn79LXAnaoFBNwAeXlVWjU6tT4zCR5tHHVwbZJcs9W7yKNAeqLcTvLCSbGZnEeMIUMjgaAtBb6G14qIFobbC7eU_3zUYkE2cs2sgGp1Mut9LAkT8SQ/mapillary_vistas_sample.zip?download&psid=1
Resolving yijzyw.am.files.1dr

In [0]:
!unzip -qq /content/mapillary_vistas_sample.zip -d /content

In [1]:
!pip install imgaug
!pip install Cython
!pip install pycocotools

In [0]:
import os
os.chdir('/content/semantic_features_detection/Notebook')

In [2]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import imageio
%tensorflow_version 1.x
import tensorflow as tf


# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Ignore Warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
print(ROOT_DIR)
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/mapillary/"))  # To find local version
import mapillary

%matplotlib inline

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
print(MODEL_DIR)

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
print('finished')

TensorFlow 1.x selected.
/content/semantic_features_detection
/content/semantic_features_detection/logs
finished


Using TensorFlow backend.


In [3]:
class TrainConfig(mapillary.mapvistas):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 30

config = TrainConfig()
config.display()
batch_size = config.BATCH_SIZE


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  640
IMAGE_META_SIZE                79
IMAGE_MIN_DIM                  640
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [640 640   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

In [4]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)
#model.load_weights(model_path, by_name=True)

# Load pretrained weights
model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead








In [5]:

TRAIN_DIR = '/content/mapillary_vistas'
# Training dataset
dataset_train = mapillary.MapillaryDataset()
dataset_train.load_vistas(dataset_dir=TRAIN_DIR, subset='training')
dataset_train.prepare()

# Validation dataset
VAL_DIR = '/content/mapillary_vistas'

dataset_val = mapillary.MapillaryDataset()
dataset_val.load_vistas(dataset_dir=VAL_DIR, subset='validation')
dataset_val.prepare()

{165: {42: {42: 0}}, 0: {0: {0: 65}}, 196: {196: {196: 2}}, 190: {255: {255: 28}}, 180: {165: {180: 4}}, 90: {120: {150: 5}}, 102: {102: {156: 6}}, 128: {64: {64: 59}}, 140: {140: {20: 51}}, 170: {170: {170: 9}}, 250: {170: {30: 48}}, 96: {96: {96: 11}}, 230: {160: {50: 26}}, 110: {110: {110: 14}}, 244: {35: {232: 15}}, 150: {0: {255: 53}}, 70: {100: {150: 43}}, 220: {220: {0: 50}}, 255: {255: {128: 32}}, 200: {128: {128: 23}}, 64: {170: {64: 25}}, 152: {251: {152: 29}}, 107: {142: {35: 30}}, 100: {128: {160: 41}}, 222: {40: {40: 37}}, 40: {40: {40: 39}}, 33: {33: {33: 40}}, 142: {0: {0: 42}}, 210: {170: {100: 44}}, 153: {153: {153: 45}}, 192: {192: {192: 49}}, 119: {11: {32: 52}}, 32: {32: {32: 63}}, 120: {10: {10: 64}}}
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65])
{1

In [0]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=MODEL_DIR, histogram_freq=1)

In [7]:
# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='heads')

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /content/semantic_features_detection/logs/mapvistas20200328T1928/mask_rcnn_mapvistas_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_m

/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/40
color 1: 196
color 2: 196
color 3: 196
color 1: 196
type of 2: <class 'numpy.uint8'>
example 1: 49
color 2: 196
color 3: 196
type of 2: <class 'numpy.uint8'>
example 1: 49
color 1: 196
color 2: 196
color 3: 196
type of 2: <class 'numpy.uint8'>
color 1: 196
example 1: 49
color 2: 196
color 3: 196
type of 2: <class 'numpy.uint8'>
example 1: 49
color 1: 190
color 2: 153
color 1: 190
color 3: 153
color 2: 153
type of 2: <class 'numpy.uint8'>
color 3: 153
type of 2: <class 'numpy.uint8'>
example 1: 49
example 1: 49


ERROR:root:Error processing image {'id': '_6lYRZS5jTuojhqapjEL1Q', 'source': 'mapillary_vistas', 'path': '/content/mapillary_vistas/validation/images/_6lYRZS5jTuojhqapjEL1Q.jpg'}
Traceback (most recent call last):
  File "/content/semantic_features_detection/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/semantic_features_detection/mrcnn/model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/content/semantic_features_detection/samples/mapillary/mapillary.py", line 153, in load_mask
    self.color_to_classid[int(class_color[0,0])][int(class_color[0,1])][int(class_color[0,2])]
KeyError: 153ERROR:root:Error processing image {'id': '_6lYRZS5jTuojhqapjEL1Q', 'source': 'mapillary_vistas', 'path': '/content/mapillary_vistas/validation/images/_6lYRZS5jTuojhqapjEL1Q.jpg'}
Traceback (most recent call last):
  File "/content/semantic_features_detection/mrcnn/model.py", line 1709, in data_generator
 

...

color 1: 196
color 2: 196
color 3: 196
type of 2: <class 'numpy.uint8'>
example 1: 49
color 1: 190
color 2: 153
color 3: 153
type of 2: <class 'numpy.uint8'>
example 1: 49


ERROR:root:Error processing image {'id': 'ZyTE16WU7OLp3yzggErRKg', 'source': 'mapillary_vistas', 'path': '/content/mapillary_vistas/training/images/ZyTE16WU7OLp3yzggErRKg.jpg'}
Traceback (most recent call last):
  File "/content/semantic_features_detection/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/semantic_features_detection/mrcnn/model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/content/semantic_features_detection/samples/mapillary/mapillary.py", line 153, in load_mask
    self.color_to_classid[int(class_color[0,0])][int(class_color[0,1])][int(class_color[0,2])]
KeyError: 153


color 1: 196
color 2: 196
color 3: 196
type of 2: <class 'numpy.uint8'>
example 1: 49
color 1: 190
color 2: 153
color 3: 153
type of 2: <class 'numpy.uint8'>
example 1: 49


ERROR:root:Error processing image {'id': 'ZyTE16WU7OLp3yzggErRKg', 'source': 'mapillary_vistas', 'path': '/content/mapillary_vistas/training/images/ZyTE16WU7OLp3yzggErRKg.jpg'}
Traceback (most recent call last):
  File "/content/semantic_features_detection/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/semantic_features_detection/mrcnn/model.py", line 1212, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/content/semantic_features_detection/samples/mapillary/mapillary.py", line 153, in load_mask
    self.color_to_classid[int(class_color[0,0])][int(class_color[0,1])][int(class_color[0,2])]
KeyError: 153


KeyError: ignored

In [0]:
# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=120,
            layers='4+')



In [0]:
# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=160,
            layers='all')